# Part IV: Models

This part of the tutorial covers model search and estimation. 

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

We will use the cholesterol data from part I as a first example to clarify the basic concepts.

In [ ]:
chol = pd.read_csv('cholesterol.csv')
chol.Time = chol.Time.astype('category')
chol.Margarine = chol.Margarine.astype('category')

The dataset is recoded so that the cholesterol measurements of each time point are available in a single row for each ID:

In [ ]:
chol_spread = chol.set_index(['ID', 'Margarine', 'Time']).unstack()
chol_spread.columns = ['T%d' % i for i in chol_spread.columns.levels[1]]
chol_spread = chol_spread.reset_index(level=1)

Categorical features need to be one-hot-encoded:

In [ ]:
chol_spread = pd.get_dummies(chol_spread)

The resulting dataset now has a column for each level of the categorical variables. `.head` can be used to inspect the structure of a DataFrame without polluting the notebook.

In [ ]:
chol_spread.head()

The standard for machine learning in python is `sklearn`. We will use it here to split the dataset into train, validation and test subsets:

In [ ]:
from sklearn.model_selection import train_test_split

ids = chol_spread.index
train_ids, test_ids = train_test_split(ids, train_size=0.75)
train_ids, validation_ids = train_test_split(train_ids, test_size = 0.3)

chol_train = chol_spread.loc[train_ids.values, :]
chol_valid = chol_spread.loc[validation_ids.values, :]
chol_test = chol_spread.loc[test_ids.values, :]

The three subsets are then always used in a very specific way.

 * The train dataset is used for fitting all models.
 * The validation dataset is used for finding the best model configuration (prediction variables, parameters, etc.).
 * The test dataset is only used to check the quality of the final model and should never be used to tune models.

In this example pipeline, we will train two different linear models using different model specifications. Both models are trained on the training dataset. In practice, the total number of models trained can of course be much larger than two, perhaps using stepwise variable selection methods.

## Regression

In this first example, we will use linear regression to predict a numeric target variable, the cholesterol measurement at time point T3.

We thus define `X` and `Y` as the predictor and target variables, respectively:

In [ ]:
X = chol_train.drop('T3', axis=1)
y = chol_train['T3']

In [ ]:
from sklearn import linear_model
from sklearn import metrics

We now define a linear regression model:

In [ ]:
m1 = linear_model.LinearRegression()

Then, we train the model on the training set:

In [ ]:
m1.fit(X, y)
m1.coef_

Now, we can compute the mean squared error function on the validation set to assess the predictive quality of the model:

In [ ]:
X_valid = chol_valid.drop('T3', axis=1)
y_valid = chol_valid['T3']
valid_pred = m1.predict(X_valid)
metrics.mean_squared_error(y_pred=valid_pred, y_true=y_valid)

Now, let's try another more complex model that includes quadratic terms in addition to the linear terms:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

m2 = Pipeline([('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
                  ('linear', linear_model.LinearRegression(fit_intercept=True))])

Again, we train a model on the training set...

In [ ]:
m2.fit(X, y)
m2.steps[1][1].coef_

...and evaluate predictive quality on the validation set:

In [ ]:
y_valid = chol_valid['T3']
valid_pred = m2.predict(X_valid)
metrics.mean_squared_error(y_pred=valid_pred, y_true=y_valid)

The validation error is much larger than for the smaller model, indicating that the complex model was overfitting. 

Based on the error estimations on the validation set, we choose to use the first model, and can now evaluate its error using the test set.

In [ ]:
X_test = chol_test.drop('T3', axis=1)
y_test = chol_test['T3']

test_pred = m1.predict(X_test)
metrics.mean_squared_error(y_pred=test_pred, y_true=y_test)

Using a separate test set to compute the generalization error when the validation error was already computed on data not previously used by the estimator might seem odd at first in this case. However, especially when more than two models are begin compared and the best one selected based on it having the lowest validation error, then this validation error is a biased estimate of the real generalization error. The selected model might just be one of several equivalent models which just happens to have the lowest validation error between them purely due to random variation.

## Classification

We will now turn to classification. This example will be based on the 2017 Stack Overflow Developer Survey dataset already used in part III. 

In [ ]:
so = pd.read_csv('survey_results_public.csv')

For this classification tast, we will look at a different subset of the dataset, to predict job satisfaction based on a number of other variables from the survey:

In [ ]:
so = pd.concat([
        so.loc[:, 'Respondent'],
        so.loc[:, 'HomeRemote':'YearsProgram'],
        so.loc[:, 'ProblemSolving':'ChangeWorld'],
        so.loc[:, ['ClickyKeys', 'Overpaid', 'TabsSpaces', 'JobSatisfaction', 'CareerSatisfaction', 'HaveWorkedLanguage']]
     ], axis=1)

Job satisfaction will be encoded as a binary variable, particioning the original answers into the categories `>= 7`, `< 7` and `NaN`.

In [ ]:
so.loc[so.JobSatisfaction[so.JobSatisfaction.notnull()].index, 'JobSatisfaction'] = so.JobSatisfaction >= 7
so.JobSatisfaction = so.JobSatisfaction.astype('category').cat.codes
so.CareerSatisfaction = so.CareerSatisfaction.astype('category')
so['UsesR'] = so.HaveWorkedLanguage.str.match('.*(R$)|.*(R;)') == True
so.drop('HaveWorkedLanguage', axis=1, inplace=True)
so.set_index('Respondent', inplace=True)

ids = so.index.unique()

Again, we split the data into train, validation and test sets...

In [ ]:
train_ids, test_ids = train_test_split(ids, train_size=0.75)
train_ids, validation_ids = train_test_split(train_ids, test_size = 0.3)

so_train = so.loc[train_ids, :]
so_valid = so.loc[validation_ids, :]
so_test = so.loc[test_ids, :]

...and differentiate between the target value `y` (`JobSatisfaction`), and the predictor variables `X`.

In [ ]:
X = so_train.drop('JobSatisfaction', axis=1)
y = so_train['JobSatisfaction']

### Logistic Regression

The first model class to be used is logistic regression, which has the advantage of relatively easy interpretability due to it being based on the theoretical framework of the linear model. We define the model object first:

In [ ]:
m1_logistic = linear_model.LogisticRegression()

Now we try to train the model on the training set:

In [ ]:
m1_logistic.fit(X, y)

This produces an error: scikit-learn cannot deal with categorical variables.
See: http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

The solution is one-hot encoding, using pandas to get_dummies:

In [ ]:
X = pd.get_dummies(so_train.drop('JobSatisfaction', axis=1), dummy_na=True)
X_valid = pd.get_dummies(so_valid.drop('JobSatisfaction', axis=1), dummy_na=True)
X_test = pd.get_dummies(so_test.drop('JobSatisfaction', axis=1), dummy_na=True)

y = so_train['JobSatisfaction']
y_valid = so_valid['JobSatisfaction']
y_test = so_test['JobSatisfaction']

We can now apply the logistic regression model to the dataset:

In [ ]:
m1_logistic = linear_model.LogisticRegression()
m1_logistic.fit(X, y)

We will try a different model using only a subset of features based on our a-priori hypothesis:

In [ ]:
cols = ['UsesR', 'TabsSpaces', 'ClickyKeys', 'Overpaid']
X_small = pd.get_dummies(so_train.drop('JobSatisfaction', axis=1)[cols], dummy_na=True)
X_valid_small = pd.get_dummies(so_valid.drop('JobSatisfaction', axis=1)[cols], dummy_na=True)
X_test_small = pd.get_dummies(so_test.drop('JobSatisfaction', axis=1)[cols], dummy_na=True)

m2_logistic = linear_model.LogisticRegression()
m2_logistic.fit(X_small, y)

We can now compare the validation accuracy of model 1...

In [ ]:
valid_pred_m1 = m1_logistic.predict(X_valid)
display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_m1))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_m1)

...and model 2:

In [ ]:
valid_pred_m2 = m2_logistic.predict(X_valid_small)
display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_m2))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_m2)

At this point, it becomes clear that the simple model is missing some crucial information: the accuracy is much lower than that of the model with all variables, not only on the training set (which could be due to overfitting), but also on the validation set.

### Random Forests

Random forests are a potentially more powerful class of models. Again, the implementation we use is from `sklearn`:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

m_rf = RandomForestClassifier()
m_rf.fit(X, y)

In [ ]:
valid_pred_rf = m_rf.predict(X_valid)

To compare it with the other models, validation set accuracy of the random forest model:

In [ ]:
display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_rf))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_rf)

First, we note that the accuracy is about the same as for glm...

Maybe this can be improved by changing one of the parameters:

In [ ]:
m_rf_2 = RandomForestClassifier(min_samples_leaf=10)
m_rf_2.fit(X, y)

valid_pred_rf_2 = m_rf_2.predict(X_valid)

We can now compute the validation set accuracy of the second random forest model:

In [ ]:
display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_rf_2))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_rf_2)

We can see that the validation accuracy has improved with the higher minimum number of samples per leaf (`min_samples_leaf`). Maybe we can try a third model with a still higher value:

In [ ]:
m_rf_3 = RandomForestClassifier(min_samples_leaf=20)
m_rf_3.fit(X, y)

valid_pred_rf_3 = m_rf_3.predict(X_valid)

display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_rf_3))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_rf_3)

Parameter tuning is not a task to be performed manually, it can be automated for a more efficient search through the parameter space. A simple tuning method for a single parameter might look as follows:

In [ ]:
def tune_rf(min_samples_leaf):
    m_rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf)
    m_rf.fit(X, y)

    valid_pred_rf = m_rf.predict(X_valid)

    return metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_rf)

In [ ]:
min_node_size_candidates = np.hstack([np.arange(1, 61, 10), np.arange(100, 1001, 300)])
pd.DataFrame(np.vstack([min_node_size_candidates, np.array([tune_rf(m) for m in min_node_size_candidates])]))

We can also try to tune multiple parameters, however, the number of models to be computed increases exponentially with the number of parameters to tune, so this method can get unwieldy rather quickly:

In [ ]:
def tune_rf(min_samples_leaf, min_samples_split):
    m_rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
    m_rf.fit(X, y)

    valid_pred_rf = m_rf.predict(X_valid)

    return metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_rf)

In [ ]:
[tune_rf(*c) for c in zip(np.repeat((20, 80, 800), 2), np.tile((2, 5), 3))]

In practice, more efficient heuristics are needed to find optimal parameters.

### Gradient Boosted Machines

The last method to be presented here, and one that is frequently found in winning entries at kaggle, is gradient 
boosting. We will use the package `xgboost` for this. (Windows users download from: https://www.lfd.uci.edu/~gohlke/pythonlibs/#xgboost)

In [ ]:
import xgboost as xgb

m_xgb = xgb.XGBClassifier()
m_xgb.fit(X, y)

Train a model and compute the validation accuracy:

In [ ]:
valid_pred_xgb = m_xgb.predict(X_valid)

display(metrics.confusion_matrix(y_true=y_valid, y_pred=valid_pred_xgb))
metrics.accuracy_score(y_true=y_valid, y_pred=valid_pred_xgb)

Only after deciding for a final model, we can use the test dataset to evaluate the accuracy of final model

In [ ]:
test_pred = m_xgb.predict(X_test)

display(metrics.confusion_matrix(y_true=y_test, y_pred=test_pred))
metrics.accuracy_score(y_true=y_test, y_pred=test_pred)